In [ ]:
import os

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(os.path.join(os.getcwd(),'Data','Amazon Search Terms_Search Terms_US.csv')).sample(n=1000,ignore_index=True)

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords
import string
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import webcolors

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
product_list = list()
for i in range(len(df['#1 Product Title'])):
    for j in range(1,4):
        product_list.append([df.loc[i,'Search Term'],df.loc[i,f'#{j} Product Title']])

In [ ]:
df = pd.DataFrame(product_list,columns=['Search Term','Product Title'])

In [ ]:
Y,X = df[['Product Title']], df[['Search Term']]

In [ ]:
Y_raw,X_raw = Y.copy(),X.copy()

In [ ]:
X_raw,Y_raw

In [ ]:
def stemSentence(sentence):
    porter = PorterStemmer()
    token_words = word_tokenize(sentence)
    stem_sentence = [porter.stem(word) for word in token_words]
    return ' '.join(stem_sentence)
def Clean_Sequence(X=X,is_X = True):
    if is_X:
        products = pd.Series(list(X['Search Term']))
    else:
        products = pd.Series(list(X['Product Title']))
    
    products = [remove_stopwords(str(x))\
            .translate(str.maketrans('','',string.punctuation))\
            .translate(str.maketrans('','',string.digits))\
            for x in products]
    
    if is_X:
        X.loc[:,'Search Term'] = products
    else:
        X.loc[:,'Product Title'] = products
    
    if is_X:
        products = pd.Series(list(X['Search Term']))
    else:
        products = pd.Series(list(X['Product Title']))
    
    products = pd.Series([stemSentence(str(x)) for x in products])
    
    if is_X:
        X.loc[:,'Search Term'] = products
    else:
        X.loc[:,'Product Title'] = products
    
    colors = list(webcolors.CSS3_NAMES_TO_HEX)
    colors = [stemSentence(str(x)) for x in colors if x not in ('bisque','blanchedalmond','chocolate','honeydew','lime',
                                             'olive','orange','plum','salmon','tomato','wheat')]

    if is_X:
        products = pd.Series(list(X['Search Term']))
    else:
        products = pd.Series(list(X['Product Title']))
    
    products = [' '.join([x for x in str(string).split() if x not in colors]) for string in products]

    if is_X:
        X.loc[:,'Search Term'] = products
    else:
        X.loc[:,'Product Title'] = products

    return X

X = Clean_Sequence(X)
Y = Clean_Sequence(Y,is_X=False)

In [ ]:
X,Y

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_productid = tfidf_vectorizer.fit_transform((Y['Product Title']))
#Y_verctorized = pd.concat([Y,pd.DataFrame(Y_verctorized.toarray(),
#                            columns=Y_verctorized.get_feature_names_out())],axis=1)

In [ ]:
def Search_system():
    Search_String = input('What to search? :')
    index = np.where(X_raw['Search Term'] == Search_String)[0][0]
    Search_q = X_raw.iloc[[index]]
    Search_q = Clean_Sequence(Search_q)
    Search_tfidf = tfidf_vectorizer.transform(Search_q['Search Term'])
    return [Search_tfidf,Search_String]

In [ ]:
def get_recommendation(top, df_all, scores, Search_String):
    recommendation = pd.DataFrame(columns = ['Search Term',  'Results', 'score'])
    count = 0
    for i in top:
        recommendation.at[count, 'Search Term'] = Search_String
        recommendation.at[count, 'Results'] = df_all['Product Title'][i]
        recommendation.at[count, 'score'] =  scores[count]
        count += 1
    return recommendation.loc[:4,]

In [ ]:
KNN = NearestNeighbors(n_neighbors = 11)
KNN.fit(tfidf_productid)
Searched = Search_system()
Search_tfidf,Search_string = Searched[0],Searched[1]
NNs = KNN.kneighbors(Search_tfidf, return_distance=True)
top = NNs[1][0][1:]
index_score = NNs[0][0][1:]
get_recommendation(top, Y_raw, index_score, Search_string)